In [4]:
# Usando dados como pontos marcados, sofridos, eficiência ofensiva/defensiva e rebotes, prever quantas vitórias uma equipa terá numa temporada

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np


# 1️⃣ Ler dados originais (sem tocar neles)
teams_df = pd.read_csv("data/teams.csv")

# 2️⃣ Criar uma cópia para manipulação
teams_model = teams_df.copy()

# 3️⃣ Criar features derivadas úteis para prever vitórias
teams_model["point_diff"] = teams_model["o_pts"] - teams_model["d_pts"]
teams_model["off_efficiency"] = teams_model["o_pts"] / teams_model["o_fga"]
teams_model["def_efficiency"] = teams_model["d_pts"] / teams_model["d_fga"]
teams_model["rebalance"] = teams_model["o_reb"] - teams_model["d_reb"]

# Remover linhas com valores nulos ou anómalos
teams_model = teams_model.dropna(subset=["won", "o_pts", "d_pts"])


teams_model.to_csv("data_models/teams_model.csv", index=False)